# Dense BR Comparison (old vs new)

Compare exploitability and runtime between `br_exact_dense_to_dense` and `br_exact_dense_new`.


In [ ]:
import os, sys, time
from pathlib import Path

def find_repo_root(start_dir: str) -> str:
    cur = os.path.abspath(start_dir)
    for _ in range(6):
        if os.path.isdir(os.path.join(cur, "liars_poker")) or os.path.exists(os.path.join(cur, "pyproject.toml")):
            return cur
        parent = os.path.dirname(cur)
        if parent == cur:
            break
        cur = parent
    return os.path.abspath(os.path.join(start_dir, "..", ".."))

NB_DIR = os.getcwd()
REPO_ROOT = find_repo_root(NB_DIR)
if REPO_ROOT not in sys.path:
    sys.path.insert(0, REPO_ROOT)

from liars_poker.serialization import load_policy
from liars_poker.policies.tabular_dense import DenseTabularPolicy
from liars_poker.algo.br_exact_dense_to_dense import best_response_dense as best_response_old
from liars_poker.algo.br_exact_dense_new import best_response_dense as best_response_new

ARTIFACTS_ROOT = Path(REPO_ROOT) / "artifacts"
RUNS_ROOT = ARTIFACTS_ROOT / "benchmark_runs"
print("repo root:", REPO_ROOT)
print("runs root:", RUNS_ROOT)


In [ ]:
def load_dense_policies(max_runs=3, max_k=12):
    runs = []
    for run_dir in sorted(RUNS_ROOT.iterdir()):
        policy_dir = run_dir / "policy"
        if not policy_dir.exists():
            continue
        try:
            pol, spec = load_policy(str(policy_dir))
        except Exception:
            continue
        if not isinstance(pol, DenseTabularPolicy):
            continue
        if pol.k > max_k:
            continue
        runs.append((run_dir.name, pol, spec))
        if len(runs) >= max_runs:
            break
    return runs

runs = load_dense_policies(max_runs=22, max_k=120)
print(f"Loaded {len(runs)} dense runs")
for name, pol, spec in runs:
    print(name, spec, f"k={pol.k}, hands={len(pol.hands)}")


In [ ]:
# Exploitability comparison
for name, pol, spec in runs:
    br_old, log_old = best_response_old(spec, pol, debug=False, store_state_values=False)
    p1_old, p2_old = log_old["computer"].exploitability()

    br_new, log_new = best_response_new(spec, pol, debug=False, store_state_values=False)
    p1_new, p2_new = log_new["computer"].exploitability()

    print(name)
    print(f" old: p1={p1_old:.6f}, p2={p2_old:.6f}")
    print(f" new: p1={p1_new:.6f}, p2={p2_new:.6f}")
    print(f" diff: dp1={abs(p1_old - p1_new):.3e}, dp2={abs(p2_old - p2_new):.3e}")
    assert abs(p1_old - p1_new) < 1e-6
    assert abs(p2_old - p2_new) < 1e-6
    print()


In [ ]:
# Runtime comparison on the first loaded run
if runs:
    name, pol, spec = runs[-1]
    print("Benchmarking:", name)

    t0 = time.perf_counter()
    best_response_old(spec, pol, debug=False, store_state_values=False)
    t1 = time.perf_counter()

    best_response_new(spec, pol, debug=False, store_state_values=False)
    t2 = time.perf_counter()

    print(f"old solver: {t1 - t0:.3f}s")
    print(f"new solver: {t2 - t1:.3f}s")
